In [1]:
#THE_FINAL_DELIVERY_NETWORK

import csv
import networkx as nx
from geopy.geocoders import Nominatim
import geopy.distance
import folium

# Define the warehouse locations
warehouse_locations = [(37.7749, -122.4194), (37.7876, -122.3962)]

# Read the delivery locations from a CSV file
delivery_locations = []
with open('/Users/nikhilks/Library/Mobile Documents/com~apple~Numbers/Documents/Final Datasets/Route Location.csv') as f:
    reader = csv.reader(f)
    next(reader)  # skip the header row
    for row in reader:
        address, product, *extra_cols = row
        delivery_locations.append({'Purchase Address': address, 'Product Name': product})

# Set a custom user_agent for Nominatim
geolocator = Nominatim(user_agent="my-app", timeout=10)

# Convert delivery addresses to coordinates
for location in delivery_locations:
    address = location['Purchase Address']
    try:
        location['coordinates'] = geolocator.geocode(address)[1]
    except:
        print(f"Could not geocode address: {address}")
        location['coordinates'] = (0, 0)  # set coordinates to default value


# Create a graph of the delivery network
G = nx.DiGraph()
for i, warehouse_location in enumerate(warehouse_locations):
    G.add_node(f'warehouse_{i}', pos=warehouse_location)
for i, location in enumerate(delivery_locations):
    G.add_node(f'delivery_{i}', pos=location['coordinates'])
    for j, warehouse_location in enumerate(warehouse_locations):
        G.add_edge(f'warehouse_{j}', f'delivery_{i}', weight=geopy.distance.distance(warehouse_location, location['coordinates']).km)
    for j in range(i + 1, len(delivery_locations)):
        dist = geopy.distance.distance(location['coordinates'], delivery_locations[j]['coordinates']).km
        G.add_edge(f'delivery_{i}', f'delivery_{j}', weight=dist)

# Find the shortest path for each product
cost_per_km = 1
for product in set([location['Product Name'] for location in delivery_locations]):
    delivery_nodes = [f'delivery_{i}' for i, location in enumerate(delivery_locations) if location['Product Name'] == product]
    paths = [nx.shortest_path(G, f'warehouse_{j}', node) for j in range(len(warehouse_locations)) for node in delivery_nodes]
    print(f'Shortest paths for {product}:')
    for i, path in enumerate(paths):
        distance = sum([G.edges[(path[j], path[j+1])]['weight'] for j in range(len(path)-1)])
        saving = (len(warehouse_locations) - 1 - i) * distance * cost_per_km  
        # calculate the savings from using a closer warehouse
        print(f'  Delivery {i}: {path}, Distance: {distance:.2f} km, Savings: ${saving:.2f}')
        
# Create a map and add markers for the warehouses and delivery locations
m = folium.Map(location=delivery_locations[0]['coordinates'], zoom_start=10)
for i, location in enumerate(warehouse_locations):
    folium.Marker(location=location, popup=f'Warehouse {i}', icon=folium.Icon(color='blue')).add_to(m)
for i, location in enumerate(delivery_locations):
    folium.Marker(location=location['coordinates'], popup=f"{location['Product Name']} Delivery {i}", icon=folium.Icon(color='red')).add_to(m)

# Add edges for the delivery network
for edge in G.edges:
    loc1 = G.nodes[edge[0]]['pos']
    loc2 = G.nodes[edge[1]]['pos']
    color = 'green' if edge[0].startswith('warehouse') else 'black'
    folium.PolyLine([loc1, loc2], color=color, weight=2, opacity=0.5).add_to(m)
    
# Display the map
m


Shortest paths for Newell 319:
  Delivery 0: ['warehouse_0', 'delivery_13'], Distance: 1.26 km, Savings: $1.26
  Delivery 1: ['warehouse_1', 'delivery_13'], Distance: 3.70 km, Savings: $0.00
Shortest paths for Fellowes Superior 10 Outlet Split Surge Protector:
  Delivery 0: ['warehouse_0', 'delivery_9'], Distance: 5.25 km, Savings: $5.25
  Delivery 1: ['warehouse_1', 'delivery_9'], Distance: 7.25 km, Savings: $0.00
Shortest paths for Cisco CP/7937G Unified IP Conference Station Phone:
  Delivery 0: ['warehouse_0', 'delivery_10'], Distance: 0.80 km, Savings: $0.80
  Delivery 1: ['warehouse_1', 'delivery_10'], Distance: 2.87 km, Savings: $0.00
Shortest paths for Tenex Carpeted, Granite/Look or Clear Contemporary Contour Shape Chair Mats:
  Delivery 0: ['warehouse_0', 'delivery_4'], Distance: 1.78 km, Savings: $1.78
  Delivery 1: ['warehouse_1', 'delivery_4'], Distance: 4.14 km, Savings: $0.00
Shortest paths for Fluorescent Highlighters by Dixon:
  Delivery 0: ['warehouse_0', 'delivery_6'